In [41]:
# 1. Import libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from pythainlp.tokenize import word_tokenize
from pythainlp.corpus import thai_stopwords
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Dropout
from sklearn.metrics import classification_report, accuracy_score
import re
from collections import Counter
import ast


In [2]:
# 2. Read the dataset
data_df = pd.read_csv('data.csv')
print("ข้อมูลต้นฉบับ:")
data_df.head()

ข้อมูลต้นฉบับ:


,url,date,title,body_text,labels
0,https://prachatai.com/print/42,2004-08-24 14:31,"นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ""",ประชาไท --- 23 ส.ค.2547 นักวิจัยฯ ชี้นโยบายจี...,"['ข่าว', 'สิ่งแวดล้อม']"
1,https://prachatai.com/print/41,2004-08-24 14:22,ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ,ประชาไท- 23 ส.ค.2547 นักวิชาการ ภาคประชาชน จ...,"['ข่าว', 'สิ่งแวดล้อม']"
2,https://prachatai.com/print/43,2004-08-24 15:17,จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย,นโยบายที่อนุญาตให้ปลูกร่วมกับพืชอื่นได้นั้นถื...,"['ข่าว', 'สิ่งแวดล้อม']"
3,https://prachatai.com/print/45,2004-08-24 15:58,ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...,ประชาไท -- 23 ส.ค. 47 ขาประจำทักษิณ ฟันธง ฟอง...,"['ข่าว', 'การเมือง', 'คณะเศรษฐศาสตร์ มหาวิทยาล..."
4,https://prachatai.com/print/47,2004-08-24 16:10,กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง,ประชาไท-23 ส.ค.47 คณะกรรมการอนุรักษ์ ผืนป่าตะ...,"['ข่าว', 'สิ่งแวดล้อม']"


In [3]:
# 3. Combine 'title' and 'body_text' into 'combined_text' and drop unnecessary columns
data_df['combined_text'] = data_df['title'] + ' ' + data_df['body_text']
data_df = data_df.drop(columns=['url', 'date', 'title', 'body_text'])
print("\nข้อมูลหลังจากรวมข้อความและลบคอลัมน์ที่ไม่จำเป็น:")
data_df.head()


ข้อมูลหลังจากรวมข้อความและลบคอลัมน์ที่ไม่จำเป็น:


,labels,combined_text
0,"['ข่าว', 'สิ่งแวดล้อม']","นักวิจัยหนุน ""แม้ว"" เปิด ""จีเอ็มโอ"" ประชาไท..."
1,"['ข่าว', 'สิ่งแวดล้อม']",ภาคประชาชนต้านเปิดเสรีจีเอ็มโอ ประชาไท- 23 ส.ค...
2,"['ข่าว', 'สิ่งแวดล้อม']",จุฬาฯ ห่วงจีเอ็มโอลามข้าวไทย นโยบายที่อนุญาตใ...
3,"['ข่าว', 'การเมือง', 'คณะเศรษฐศาสตร์ มหาวิทยาล...",ฟองสบู่การเมืองแตก ทักษิณหมดกึ๋น ชนชั้นกลางหมด...
4,"['ข่าว', 'สิ่งแวดล้อม']",กอต.เสนอเลิกถนนคลองลาน-อุ้มผาง ประชาไท-23 ส.ค....


In [4]:
# 4. Count individual label frequencies and filter labels with at least min_label_count
# แปลงคอลัมน์ 'labels' จาก string เป็น list ถ้ายังไม่ได้ทำ
data_df['labels'] = data_df['labels'].apply(ast.literal_eval)

# ตรวจสอบว่าคอลัมน์ 'labels' เป็น list จริง ๆ
print("\nตรวจสอบชนิดข้อมูลในคอลัมน์ 'labels':")
print(data_df['labels'].iloc[:5])



ตรวจสอบชนิดข้อมูลในคอลัมน์ 'labels':
0                                  [ข่าว, สิ่งแวดล้อม]
1                                  [ข่าว, สิ่งแวดล้อม]
2                                  [ข่าว, สิ่งแวดล้อม]
3    [ข่าว, การเมือง, คณะเศรษฐศาสตร์ มหาวิทยาลัยธรร...
4                                  [ข่าว, สิ่งแวดล้อม]
Name: labels, dtype: object


In [5]:
# ตรวจสอบว่าทุกแถวเป็น list ของ string
print("\nตัวอย่างข้อมูลในคอลัมน์ 'labels' หลังการแปลง:")
print(data_df['labels'].iloc[:10])



ตัวอย่างข้อมูลในคอลัมน์ 'labels' หลังการแปลง:
0                                  [ข่าว, สิ่งแวดล้อม]
1                                  [ข่าว, สิ่งแวดล้อม]
2                                  [ข่าว, สิ่งแวดล้อม]
3    [ข่าว, การเมือง, คณะเศรษฐศาสตร์ มหาวิทยาลัยธรร...
4                                  [ข่าว, สิ่งแวดล้อม]
5                                  [ข่าว, สิ่งแวดล้อม]
6    [ข่าว, การเมือง, ชนชั้นกลาง, ทักษิณ ชินวัตร, ท...
7                                  [ข่าว, สิ่งแวดล้อม]
8                                  [ข่าว, สิ่งแวดล้อม]
9                                             [บทความ]
Name: labels, dtype: object


In [6]:
# ตรวจสอบและแก้ไขป้ายกำกับที่เป็น nested lists
def flatten_labels(labels):
    if isinstance(labels, list):
        return [item for sublist in labels for item in (sublist if isinstance(sublist, list) else [sublist])]
    return []

data_df['labels'] = data_df['labels'].apply(flatten_labels)

# ตรวจสอบอีกครั้ง
print("\nตัวอย่างข้อมูลในคอลัมน์ 'labels' หลังการ flatten:")
print(data_df['labels'].iloc[:10])



ตัวอย่างข้อมูลในคอลัมน์ 'labels' หลังการ flatten:
0                                  [ข่าว, สิ่งแวดล้อม]
1                                  [ข่าว, สิ่งแวดล้อม]
2                                  [ข่าว, สิ่งแวดล้อม]
3    [ข่าว, การเมือง, คณะเศรษฐศาสตร์ มหาวิทยาลัยธรร...
4                                  [ข่าว, สิ่งแวดล้อม]
5                                  [ข่าว, สิ่งแวดล้อม]
6    [ข่าว, การเมือง, ชนชั้นกลาง, ทักษิณ ชินวัตร, ท...
7                                  [ข่าว, สิ่งแวดล้อม]
8                                  [ข่าว, สิ่งแวดล้อม]
9                                             [บทความ]
Name: labels, dtype: object


In [7]:
# แปลงแต่ละป้ายกำกับให้อยู่ในแถวแยกกัน
labels_exploded = data_df.explode('labels')

# นับความถี่ของแต่ละป้ายกำกับ
label_counts = labels_exploded['labels'].value_counts()
print("\nจำนวนแต่ละ label ก่อนการกรอง:")
print(label_counts)



จำนวนแต่ละ label ก่อนการกรอง:
labels
ข่าว                      53448
การเมือง                  39124
สิทธิมนุษยชน              15067
คุณภาพชีวิต               11316
บทความ                     9571
                          ...  
เทวาลัย                       1
เครื่องฉีดน้ำแรงดันสูง        1
อักษรศาสตร์                   1
รอยพญานาค                     1
วันประกาศเอกราช               1
Name: count, Length: 51797, dtype: int64


In [8]:
desired_labels = ['ข่าว', 'สิ่งแวดล้อม', 'บทความ', 'การเมือง','คุณภาพชีวิต','สิทธิมนุษยชน']  # ปรับตามความต้องการของคุณ


In [9]:
# กำหนดค่าต่ำสุดสำหรับจำนวน labels
# min_label_count = 50  # คุณสามารถปรับค่าตามต้องการ

# กำหนด labels ที่จะเก็บ
labels_to_keep = [label for label in label_counts.index if label in desired_labels]
print("\nLabels ที่จะเก็บ:")
print(labels_to_keep)


Labels ที่จะเก็บ:
['ข่าว', 'การเมือง', 'สิทธิมนุษยชน', 'คุณภาพชีวิต', 'บทความ', 'สิ่งแวดล้อม']


In [10]:
unwanted_labels = [
    'ใบตองแห้ง\' ออนไลน์', '10 เมษายน 2553', '112', '14 ตุลาคม 2516', '15 พฤษภาคม 2553',
    '19 กันยายน 2549', '19 พฤษภาคม 2553', '1984', '24 มิถุนาประชาธิปไตย',
    '24 มิถุนายน 2475', '30 บาทรักษาทุกโรค', '3G', '40 ปี 14 ตุลาคม 2516',
    '40 ปี 6 ตุลาคม 2519', '6 ตุลาคม 2519', '6 ศพวัดปทุม', 'AEC', 'ASEAN',
    'Arab Spring', 'Facebook', 'ISIS', 'TCIJ', 'TDRI', 'BRN', 'EHIA', 'EIA',
    'LGBT', 'NSA'  # เพิ่มป้ายกำกับที่ต้องการลบ
]

In [11]:
# ลบ unwanted_labels ออกจาก labels_to_keep
labels_to_keep = [label for label in labels_to_keep if label not in unwanted_labels]
print("\nLabels ที่จะเก็บหลังจากลบ unwanted_labels:")
print(labels_to_keep)


Labels ที่จะเก็บหลังจากลบ unwanted_labels:
['ข่าว', 'การเมือง', 'สิทธิมนุษยชน', 'คุณภาพชีวิต', 'บทความ', 'สิ่งแวดล้อม']


In [12]:
# กรองข้อมูลเพื่อลบ labels ที่ไม่มีใน labels_to_keep และลบ NaN และ labels ที่เป็น []
filtered_data = data_df.copy()  # ใช้ .copy() เพื่อหลีกเลี่ยง SettingWithCopyWarning

# ฟังก์ชันสำหรับกรองป้ายกำกับในแต่ละแถว
def filter_labels(labels, labels_to_keep):
    return [label for label in labels if label in labels_to_keep]

filtered_data['labels'] = filtered_data['labels'].apply(lambda x: filter_labels(x, labels_to_keep))

# ลบแถวที่ไม่มีป้ายกำกับเหลือ
filtered_data = filtered_data[filtered_data['labels'].map(len) > 0]
print(f"\nขนาดของ filtered_data: {filtered_data.shape}")



ขนาดของ filtered_data: (67106, 2)


In [13]:
# ตรวจสอบจำนวนแต่ละ label หลังการกรอง
labels_exploded_filtered = filtered_data.explode('labels')
filtered_label_counts = labels_exploded_filtered['labels'].value_counts()
print("\nจำนวนแต่ละ label หลังการกรอง:")
print(filtered_label_counts)



จำนวนแต่ละ label หลังการกรอง:
labels
ข่าว            53448
การเมือง        39124
สิทธิมนุษยชน    15067
คุณภาพชีวิต     11316
บทความ           9571
สิ่งแวดล้อม      7703
Name: count, dtype: int64


In [14]:
# 5. Tokenization
filtered_data['tokenized_text'] = filtered_data['combined_text'].apply(lambda x: ' '.join(word_tokenize(x, engine='newmm')))

In [15]:
# 6. Cleaning text
stopwords = list(thai_stopwords())

def clean_text(text):
    # ลบสัญลักษณ์พิเศษ, ตัวเลข, และเครื่องหมาย ':' ออกจากข้อความ
    text = re.sub(r'[\"\'\(\)\-\.!/,0-9:]', '', text)
    # ลบ stop words
    return ' '.join([word for word in text.split() if word not in stopwords])

filtered_data['cleaned_text'] = filtered_data['tokenized_text'].apply(clean_text)


In [16]:
# 7. Drop unnecessary columns
filtered_data = filtered_data.drop(columns=['combined_text', 'tokenized_text'])
filtered_data.head()

,labels,cleaned_text
0,"[ข่าว, สิ่งแวดล้อม]",นักวิจัย หนุน แม้ว จีเอ็มโอ ประชา ไท สค นักวิจ...
1,"[ข่าว, สิ่งแวดล้อม]",ประชาชน ต้าน เสรี จีเอ็มโอ ประชา ไท สค นักวิชา...
2,"[ข่าว, สิ่งแวดล้อม]",จุฬาฯ ห่วง จีเอ็มโอ ลาม ข้าว ไทย นโยบาย อนุญาต...
3,"[ข่าว, การเมือง]",ฟองสบู่ การเมือง แตก ทักษิณ กึ๋น ชนชั้นกลาง คว...
4,"[ข่าว, สิ่งแวดล้อม]",กอ ต เสนอ เลิก ถนน คลอง ลาน อุ้มผาง ประชา ไท ส...


In [17]:
# 8. Word frequency analysis
all_words = ' '.join(filtered_data['cleaned_text']).split()
word_counts = Counter(all_words)
sorted_word_counts = word_counts.most_common(40)

print("\n20 คำที่ปรากฏบ่อยที่สุด:")
for word, count in sorted_word_counts[:20]:
    print(f"'{word}': {count} ครั้ง")

# แสดงคำที่ถูกละเลยบางส่วน (ถ้ามี)
print("\nคำที่ถูกละเลยบางส่วน:")
for word, count in sorted_word_counts[20:]:
    print(f"'{word}': {count} ครั้ง")


20 คำที่ปรากฏบ่อยที่สุด:
'คน': 366335 ครั้ง
'ไทย': 291302 ครั้ง
'เรื่อง': 223800 ครั้ง
'ประชาชน': 217717 ครั้ง
'ปี': 215021 ครั้ง
'ประเทศ': 194616 ครั้ง
'รัฐบาล': 177733 ครั้ง
'ทำ': 160155 ครั้ง
'”': 151803 ครั้ง
'“': 150086 ครั้ง
'พื้นที่': 137762 ครั้ง
'ที่จะ': 135618 ครั้ง
'รัฐ': 131674 ครั้ง
'วันที่': 128809 ครั้ง
'ผม': 124468 ครั้ง
'สังคม': 124310 ครั้ง
'สิทธิ': 110628 ครั้ง
'สร้าง': 108415 ครั้ง
'แรงงาน': 94838 ครั้ง
'[]': 93645 ครั้ง

คำที่ถูกละเลยบางส่วน:
'กรณี': 92212 ครั้ง
'จังหวัด': 91910 ครั้ง
'กฎหมาย': 91671 ครั้ง
'ระบบ': 84480 ครั้ง
'ปัญหา': 83906 ครั้ง
'ชุมนุม': 81569 ครั้ง
'รัฐธรรมนูญ': 78003 ครั้ง
'ประชาธิปไตย': 77299 ครั้ง
'บาท': 76717 ครั้ง
'ทหาร': 75474 ครั้ง
'ที่ผ่านมา': 75373 ครั้ง
'เกิดขึ้น': 72216 ครั้ง
'จำนวน': 71195 ครั้ง
'ชาวบ้าน': 70965 ครั้ง
'กล่าวว่า': 69898 ครั้ง
'ชุมชน': 68605 ครั้ง
'ดี': 68490 ครั้ง
'ระบุ': 68443 ครั้ง
'รายงาน': 68215 ครั้ง
'ข้อมูล': 68172 ครั้ง


In [18]:
print("\nLabels ที่จะเก็บหลังการกรอง:")
print(labels_to_keep)


Labels ที่จะเก็บหลังการกรอง:
['ข่าว', 'การเมือง', 'สิทธิมนุษยชน', 'คุณภาพชีวิต', 'บทความ', 'สิ่งแวดล้อม']


In [19]:
print("\nตัวอย่างข้อมูลในคอลัมน์ 'labels' หลังการแปลงและกรอง:")
print(filtered_data['labels'].iloc[:5])


ตัวอย่างข้อมูลในคอลัมน์ 'labels' หลังการแปลงและกรอง:
0    [ข่าว, สิ่งแวดล้อม]
1    [ข่าว, สิ่งแวดล้อม]
2    [ข่าว, สิ่งแวดล้อม]
3       [ข่าว, การเมือง]
4    [ข่าว, สิ่งแวดล้อม]
Name: labels, dtype: object


In [20]:
print("\nจำนวนแต่ละ label หลังการกรอง:")
print(filtered_label_counts)


จำนวนแต่ละ label หลังการกรอง:
labels
ข่าว            53448
การเมือง        39124
สิทธิมนุษยชน    15067
คุณภาพชีวิต     11316
บทความ           9571
สิ่งแวดล้อม      7703
Name: count, dtype: int64


In [21]:
filtered_data.head(11)

,labels,cleaned_text
0,"[ข่าว, สิ่งแวดล้อม]",นักวิจัย หนุน แม้ว จีเอ็มโอ ประชา ไท สค นักวิจ...
1,"[ข่าว, สิ่งแวดล้อม]",ประชาชน ต้าน เสรี จีเอ็มโอ ประชา ไท สค นักวิชา...
2,"[ข่าว, สิ่งแวดล้อม]",จุฬาฯ ห่วง จีเอ็มโอ ลาม ข้าว ไทย นโยบาย อนุญาต...
3,"[ข่าว, การเมือง]",ฟองสบู่ การเมือง แตก ทักษิณ กึ๋น ชนชั้นกลาง คว...
4,"[ข่าว, สิ่งแวดล้อม]",กอ ต เสนอ เลิก ถนน คลอง ลาน อุ้มผาง ประชา ไท ส...
5,"[ข่าว, สิ่งแวดล้อม]",สำรวจ เส้นทาง ถนน สาย มั่ง คลอง ลาน อุ้มผาง ภา...
6,"[ข่าว, การเมือง]",ชนชั้นกลาง ความหมาย เกษียร เต ชะ พี ระ รศ เกษี...
7,"[ข่าว, สิ่งแวดล้อม]",ปี เส้นทาง สาย โครงการ ถนน สาย คลอง ลาน อุ้มผา...
8,"[ข่าว, สิ่งแวดล้อม]",เบนซิน ขยับ บาท ลิตร กรุงเทพฯ สค กระทรวง พลังง...
9,[บทความ],บทวิเคราะห์ ความสัมพันธ์ บรรษัทข้ามชาติ สื่อ ร...


In [22]:
# ตรวจสอบจำนวนแต่ละ label หลังการกรอง
labels_exploded_filtered = filtered_data.explode('labels')
filtered_label_counts = labels_exploded_filtered['labels'].value_counts()
print("\nจำนวนแต่ละ label หลังการกรอง:")
print(filtered_label_counts)



จำนวนแต่ละ label หลังการกรอง:
labels
ข่าว            53448
การเมือง        39124
สิทธิมนุษยชน    15067
คุณภาพชีวิต     11316
บทความ           9571
สิ่งแวดล้อม      7703
Name: count, dtype: int64


In [23]:
import pandas as pd
filter_labels.head()

AttributeError: 'function' object has no attribute 'head'

In [24]:
# 9. One-Hot Encoding ของป้ายกำกับ
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
encoded_labels = mlb.fit_transform(filtered_data['labels'])
print("ป้ายกำกับที่แปลงเป็น One-Hot Encoding:")
print(mlb.classes_)


ป้ายกำกับที่แปลงเป็น One-Hot Encoding:
['การเมือง' 'ข่าว' 'คุณภาพชีวิต' 'บทความ' 'สิทธิมนุษยชน' 'สิ่งแวดล้อม']


In [25]:
# 10. เตรียมข้อมูลสำหรับการฝึกโมเดล
max_words = 20000  # เลือก 20,000 คำ
tokenizer = Tokenizer(num_words=max_words, oov_token="<OOV>")
tokenizer.fit_on_texts(filtered_data['cleaned_text'])
sequences = tokenizer.texts_to_sequences(filtered_data['cleaned_text'])

max_len = 500  # ความยาวสูงสุดของข้อความ
data = pad_sequences(sequences, maxlen=max_len)
print("\nตัวอย่างการแปลงเป็น sequences:")
print(data[:5])


ตัวอย่างการแปลงเป็น sequences:
[[   0    0    0 ... 2167  133  288]
 [   0    0    0 ... 2167  133  288]
 [   0    0    0 ... 2167  133  288]
 [   0    0    0 ... 2167  133  288]
 [   0    0    0 ... 2167  133  288]]


In [26]:
# 11. แบ่งข้อมูลเป็นชุดฝึกและทดสอบ
X_train, X_test, y_train, y_test = train_test_split(data, encoded_labels, test_size=0.2, random_state=42)
print(f"\nขนาดของชุดฝึก: {X_train.shape}, ขนาดของชุดทดสอบ: {X_test.shape}")



ขนาดของชุดฝึก: (53684, 500), ขนาดของชุดทดสอบ: (13422, 500)


In [27]:
# 12. สร้างโมเดล CNN สำหรับ Multi-label Classification
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Conv1D, MaxPooling1D, Flatten, Dense, Dropout

model = Sequential()
model.add(Embedding(input_dim=max_words + 1, output_dim=128, input_length=max_len))
model.add(Conv1D(filters=32, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(mlb.classes_), activation='sigmoid'))  # ใช้ sigmoid สำหรับ multi-label

# แสดงโครงสร้างโมเดล
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          2560128   
                                                                 
 conv1d (Conv1D)             (None, 498, 32)           12320     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 249, 32)          0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 7968)              0         
                                                                 
 dense (Dense)               (None, 64)                510016    
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                        

In [28]:
# 13. คอมไพล์โมเดล
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [29]:
# 14. ฝึกโมเดล
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_loss')

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping, model_checkpoint])


Epoch 1/10


2024-10-09 08:10:33.603927: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


1343/1343 [==============================] - 21s 16ms/step - loss: 0.3928 - accuracy: 0.4486 - val_loss: 0.3178 - val_accuracy: 0.4381
Epoch 2/10
1343/1343 [==============================] - 22s 16ms/step - loss: 0.3106 - accuracy: 0.4598 - val_loss: 0.3095 - val_accuracy: 0.4472
Epoch 3/10
1343/1343 [==============================] - 21s 15ms/step - loss: 0.2636 - accuracy: 0.4666 - val_loss: 0.3176 - val_accuracy: 0.4567
Epoch 4/10
1343/1343 [==============================] - 20s 15ms/step - loss: 0.2238 - accuracy: 0.4741 - val_loss: 0.3651 - val_accuracy: 0.4131
Epoch 5/10
1343/1343 [==============================] - 19s 14ms/step - loss: 0.1910 - accuracy: 0.4853 - val_loss: 0.3985 - val_accuracy: 0.4098


In [30]:
# 15. ทำนายผล
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)  # แปลงค่าความน่าจะเป็นเป็น 0 หรือ 1


420/420 [==============================] - 1s 3ms/step


In [31]:
# 16. ประเมินผลโมเดล
from sklearn.metrics import classification_report, accuracy_score

# คำนวณ Macro Accuracy
macro_accuracy = accuracy_score(y_test, y_pred_classes)
print(f'\nMacro Accuracy: {macro_accuracy}')

# สร้าง Classification Report
print("\nClassification Report:")
print(classification_report(y_test, y_pred_classes, target_names=mlb.classes_, zero_division=0))



Macro Accuracy: 0.5146028907763374

Classification Report:
              precision    recall  f1-score   support

    การเมือง       0.82      0.82      0.82      7776
        ข่าว       0.89      0.96      0.92     10669
 คุณภาพชีวิต       0.78      0.37      0.50      2256
      บทความ       0.76      0.48      0.59      1921
สิทธิมนุษยชน       0.79      0.39      0.52      2980
 สิ่งแวดล้อม       0.79      0.55      0.65      1578

   micro avg       0.84      0.75      0.80     27180
   macro avg       0.80      0.60      0.67     27180
weighted avg       0.83      0.75      0.78     27180
 samples avg       0.84      0.77      0.78     27180



In [ ]:
def predict_new_text(new_text):
    tokenized_text = ' '.join(word_tokenize(new_text, engine='newmm'))
    cleaned_text = clean_text(tokenized_text)  # ใช้ฟังก์ชันทำความสะอาด
    sequence = tokenizer.texts_to_sequences([cleaned_text])  # แปลงเป็นลำดับตัวเลข
    padded_sequence = pad_sequences(sequence, maxlen=max_len)  # ทำ padding
    prediction = model.predict(padded_sequence)  # ทำนาย
    predicted_classes = (prediction > 0.5).astype(int)  # เปลี่ยนค่าความน่าจะเป็นเป็น 0 หรือ 1
    return mlb.inverse_transform(predicted_classes)  # คืนค่าชื่อ label

# ตัวอย่างการใช้งาน
new_text = ""
print(predict_new_text(new_text))

1/1 [==============================] - 0s 15ms/step
[('ข่าว',)]
